<a href="https://colab.research.google.com/github/pavlyk/DataScience/blob/master/plotly_express.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
!pip install plotly_express

In [0]:
import pandas as pd
import numpy as np
import plotly_express as px

# Suicide Data
suicides = pd.read_csv("drive/My Drive/master.csv")
del suicides['HDI for year']
del suicides['country-year']

# Country ISO Codes
iso_country_map = pd.read_csv("drive/My Drive/wikipedia-iso-country-codes.csv")
iso_country_map = iso_country_map.rename(columns = {'English short name lower case':"country"})

# Load Country Continents file
concap =pd.read_csv("drive/My Drive/countryContinent.csv", encoding='iso-8859-1')[['code_3', 'continent', 'sub_region']]
concap = concap.rename(columns = {'code_3':"Alpha-3 code"})

correct_names = {'Cabo Verde': 'Cape Verde', 'Macau': 'Macao', 'Republic of Korea': "Korea, Democratic People's Republic of" , 
 'Russian Federation': 'Russia',
 'Saint Vincent and Grenadines':'Saint Vincent and the Grenadines' 
 , 'United States': 'United States Of America'}

def correct_country(x):
    if x in correct_names:
        return correct_names[x]
    else:
        return x

suicides['country'] = suicides['country'].apply(lambda x : correct_country(x))

suicides = pd.merge(suicides,iso_country_map,on='country',how='left')
suicides = pd.merge(suicides,concap,on='Alpha-3 code',how='left')

suicides['gdp'] = suicides['gdp_per_capita ($)']*suicides['population']

In [0]:
suicides_gby_Continent = suicides.groupby(['continent','sex','year']).aggregate(np.sum).reset_index()

suicides_gby_Continent['gdp_per_capita ($)'] = suicides_gby_Continent['gdp']/suicides_gby_Continent['population']

suicides_gby_Continent['suicides/100k pop'] = suicides_gby_Continent['suicides_no']*1000/suicides_gby_Continent['population']

# 2016 data is not full
suicides_gby_Continent=suicides_gby_Continent[suicides_gby_Continent['year']!=2016]

suicides_gby_Continent.head()

,continent,sex,year,suicides_no,population,suicides/100k pop,gdp_per_capita ($),Numeric code,gdp
0,Africa,female,1985,37,470300,0.078673,1328.206889,7020,624655700
1,Africa,female,1986,8,478100,0.016733,1755.988496,7020,839538100
2,Africa,female,1987,53,487000,0.108830,2205.994251,7020,1074319200
3,Africa,female,1988,56,464900,0.120456,2317.000000,2880,1077173300
4,Africa,female,1989,37,470000,0.078723,2349.000000,2880,1104030000


In [0]:
suicides_gby_Continent_2007 = suicides_gby_Continent[suicides_gby_Continent['year']==2007]

px.scatter(suicides_gby_Continent_2007,x = 'suicides/100k pop', y = 'gdp_per_capita ($)')

In [0]:
px.scatter(suicides_gby_Continent_2007,x = 'suicides/100k pop', y = 'gdp_per_capita ($)',color='continent')

In [0]:
px.scatter(suicides_gby_Continent,x = 'suicides/100k pop', y = 'gdp_per_capita ($)',color='continent',
               size='suicides/100k pop',symbol='sex',animation_frame='year', animation_group='continent',range_x = [0,0.6],
              range_y = [0,70000],text='continent')

In [0]:
european_suicide_data = suicides[suicides['continent'] =='Europe']
european_suicide_data_gby = european_suicide_data.groupby(['age','sex','year']).aggregate(np.sum).reset_index()
european_suicide_data_gby['suicides/100k pop'] = european_suicide_data_gby['suicides_no']*1000/european_suicide_data_gby['population']


# A single line to create an animated Bar chart too.
px.bar(european_suicide_data_gby,x='age',y='suicides/100k pop',facet_col='sex',animation_frame='year', 
       animation_group='age', 
       category_orders={'age':['5-14 years', '15-24 years', '25-34 years', '35-54 years', 
       '55-74 years', '75+ years']},range_y=[0,1])

In [0]:
suicides_map = suicides.groupby(['year','country','Alpha-3 code']).aggregate(np.sum).reset_index()[['country','Alpha-3 code','suicides_no','population','year']]

suicides_map["suicides/100k pop"]=suicides_map["suicides_no"]*1000/suicides_map["population"]

px.choropleth(suicides_map, locations="Alpha-3 code", color="suicides/100k pop", hover_name="country", animation_frame="year",
             color_continuous_scale=px.colors.sequential.Plasma)